# Bert2Bert for Cnn/daily

## Libraries and environment preparation

In [1]:
#Install essential packages
%%capture
!pip install datasets rouge-score nltk wandb
!pip install transformers==4.11.0

In [2]:
#install Git-LFS
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 2,129 kB of archives.
After this operation, 7,662 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 git-lfs amd64 2.3.4-1 [2,129 kB]
Fetched 2,129 kB in 1s (2,395 kB/s)
Selecting previously unselected package git-lfs.
(Reading database ... 155222 files and directories currently installed.)
Preparing to unpack .../git-lfs_2.3.4-1_amd64.deb ...
Unpacking git-lfs (2.3.4-1) ...
Setting up git-lfs (2.3.4-1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [3]:
#Colab Environment Check for GPU and RAM
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

#GPU check
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Your runtime has 27.3 gigabytes of available RAM

Fri Nov 26 23:28:19 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                      

Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version:

In [4]:
# Make sure your version of Transformers is at 4.11.0
# to run the following code correctly:
import datasets
import transformers

In [5]:
# Import Wandb 
import os
import wandb
API_KEY = '39991c538626bee25c64d4f8a4c3403dd635537c'
os.environ["WANDB_API_KEY"] = API_KEY

## Loading the dataset

In [6]:
# import dataset and metrics with huggingface
raw_datasets = datasets.load_dataset('cnn_dailymail', '3.0.0')

Downloading:   0%|          | 0.00/3.51k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/159M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/376M [00:00<?, ?B/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [8]:
# Visualize the Data

import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=3):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    del df["id"]
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [9]:
show_random_elements(raw_datasets["train"])

,article,highlights
0,"Ibrahim Hassan al-Asiri, Al Qaeda's chief bombmaker, is believed to have been killed in an ambush by U.S- backed special forces in Yemen . An Al Qaeda mastermind whose plots included a notorious aircraft underpants bomb is believed to have been killed by special forces in Yemen. Ibrahim al-Asiri, who was also suspected of concealing explosives in ink cartridges found on planes at East Midlands Airport, is said to have died in a shootout. Officials in Yemen said they were ‘confident’ the terror chief, who specialised in building devices for mid-air attacks, was one of the victims of the ambush. Efforts were yesterday being made to identify the body but, if confirmed, the strike will be the most high-profile against Al Qaeda leaders since Osama bin Laden and hate cleric Anwar al-Awlaki were killed in 2011. Saudi Arabian-born al-Asiri, 32, is believed to have made the underpants bomb carried by one-time British student Umar Farouk Abdulmutallab in his attempt to blow up an airliner over the US on Christmas Day 2009. A US official described the target of the ambush as ‘high-value’. Al-Asiri is one of the CIA’s top two targets in the region. The other extremist heading the hit-list is Nasir al-Wuhayshi, leader of Al Qaeda in the Arabian Peninsula (AQAP) and a former member of bin Laden’s inner circle. A Yemeni government official said: ‘The body of a high-value Al Qaeda target is being tested to confirm its identity.’ The shooting, on Sunday night, was part of a three-day operation against AQAP. At least 65 militants were killed in the offensive involving CIA-controlled drone strikes, bombing raids by Yemeni jets and missions by US and Yemeni special forces. The raids followed the release of a video showing al-Wuhayshi urging a large group of extremists at an open-air meeting to ‘eliminate’ America. Scroll down for video . People gather near a destroyed car that was carrying militants in the Sawmaa area of al-Bayda province, Yemen, Saturday . At least 14 people, including 12 al-Qaeda suspects, were killed on Saturday in a drone raid in the central Yemeni province of Bayda, Yemeni media reports said . The ambush took place on a road in the . lawless southern Shabwa province, east of the capital Sanaa. Al-Asiri . is Al Qaeda’s most notorious bombmaker and is the prime suspect in . several airliner plots. He . was named as the extremist behind a plot by foiled suicide bomber . Abdulmutallab, a Nigerian former student at University College London, . who tried to detonate explosives on a Northwest Airlines flight that was . en route from Amsterdam to Detroit in 2009. The . bomb was designed to be carried within the fabric of his underwear. It . contained no metallic parts, making it extremely hard to detect. After . this plot failed, al-Asiri tried in 2010 to conceal Semtex-style . explosives in Hewlett-Packard printer ink cartridges placed on a UPS . Airline cargo flight by two Yemeni women. They were discovered at East Midlands Airport and removed from a plane en route to Chicago. On Saturday and Sunday, several air strikes - presumed to be carried out chiefly by U.S. drones - were launched on central and southern provinces of Yemen . Al-Asiri . also built a bomb for his own brother Abdullah, 23, so he could carry . out a daring attack on Saudi prince Mohammed bin Nayef bin Abdul Aziz, . the minister who led a programme to encourage jihadists to reform. Abdullah . was granted a Ramadan audience with the minister but when the device . hidden inside his body was triggered, he died and Prince Mohammed . survived. Al-Asiri is also . feared to have trained up to 30 British fanatics in Yemen, where MI6 and . MI5 believe fanatics pose a ‘real threat’ to the UK.","Ibrahim al-Asiri thought to be dead - most senior member since Bin Laden .\nAl-Asiri believed to be behind explosive devices including pant bombs .\nComes after 'U.S.' backed forces engaged vehicle in gun battle .\nThose inside managed to shoot back, but all i

## Preprocessing the data

In [10]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
tokenizer.bos_token = tokenizer.cls_token
tokenizer.eos_token = tokenizer.sep_token

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:337: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


**Following steps indentity the length of the article**

We can see that on average an article contains 848 tokens with *ca.* 3/4 of the articles being longer than the model's `max_length` 512. The summary is on average 57 tokens long. Over 30% of our 10000-sample summaries are longer than 64 tokens, but none are longer than 128 tokens.

`bert-base-cased` is limited to 512 tokens, which means we would have to cut possibly important information from the article. Because most of the important information is often found at the beginning of articles and because we want to be computationally efficient, we decide to stick to `bert-base-cased` with a `max_length` of 512 in this notebook. This choice is not optimal but has shown to yield [good results](https://arxiv.org/abs/1907.12461) on CNN/Dailymail. 

In [11]:
encoder_max_length=512
decoder_max_length=128

def process_data_to_model_inputs(batch):
  # tokenize the inputs and labels
  inputs = tokenizer(batch["article"], padding="max_length", truncation=True, max_length=encoder_max_length)
  outputs = tokenizer(batch["highlights"], padding="max_length", truncation=True, max_length=decoder_max_length)

  batch["input_ids"] = inputs.input_ids
  batch["attention_mask"] = inputs.attention_mask
  batch["decoder_input_ids"] = outputs.input_ids
  batch["decoder_attention_mask"] = outputs.attention_mask
  batch["labels"] = outputs.input_ids.copy()

  # because BERT automatically shifts the labels, the labels correspond exactly to `decoder_input_ids`. 
  # We have to make sure that the PAD token is ignored
  batch["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]]

  return batch

In [12]:
train_data = raw_datasets["train"].select(range(50000))
train_data = train_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    remove_columns=["article", "highlights", "id"]
)
train_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

  0%|          | 0/50 [00:00<?, ?ba/s]

In [13]:
# bert2bert validation step is high computation cost. We choose about 1/5
val_data = raw_datasets['validation'].select(range(2500))
val_data = val_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    remove_columns=["article", "highlights", "id"]
)
val_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

  0%|          | 0/3 [00:00<?, ?ba/s]

## Fine-tuning the model

In [14]:
# Import tokenizer from model checkpoint and print detail
from transformers import EncoderDecoderModel

bert2bert = EncoderDecoderModel.from_encoder_decoder_pretrained("bert-base-uncased", "bert-base-uncased")

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:337: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.bias', 'cls.seq_relations

Setting the special tokens.
`bert-base-cased` does not have a `decoder_start_token_id` or `eos_token_id`, so we will use its `cls_token_id` and `sep_token_id` respectively. 
Also, we should define a `pad_token_id` on the config and make sure the correct `vocab_size` is set.

In [15]:
# set special tokens
bert2bert.config.decoder_start_token_id = tokenizer.cls_token_id
bert2bert.config.eos_token_id = tokenizer.sep_token_id
bert2bert.config.pad_token_id = tokenizer.pad_token_id

# sensible parameters for beam search
bert2bert.config.vocab_size = bert2bert.config.decoder.vocab_size
bert2bert.config.max_length = 142
bert2bert.config.min_length = 56
bert2bert.config.no_repeat_ngram_size = 3
bert2bert.config.early_stopping = True
bert2bert.config.length_penalty = 2.0
bert2bert.config.num_beams = 4

Define `Seq2SeqTrainer` to compute the metrics from the predictions, and also do a bit of pre-processing to decode the predictions into texts:

In [16]:
# keep track with wandb
wandb.init(project="bert2bert-cnn")

wandb: Currently logged in as: shusunny (use `wandb login --relogin` to force relogin)


In [17]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

In [18]:
import nltk
import numpy as np
nltk.download('punkt')

from datasets import load_metric
metric = load_metric("rouge")

def compute_metrics(eval_pred):
    predictions = eval_pred.predictions
    labels = eval_pred.label_ids
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
        
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    return {k: round(v, 4) for k, v in result.items()}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Downloading:   0%|          | 0.00/2.17k [00:00<?, ?B/s]

In [19]:
# set training arguments - these params are not really tuned, feel free to change
batch_size = 8
epochs = 2
training_args = Seq2SeqTrainingArguments(
    output_dir="./",
    evaluation_strategy="steps",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    predict_with_generate=True,
    logging_steps=1000,  # set to 1000 for full training
    save_steps=500,  # set to 500 for full training
    eval_steps=2500,  # set to 8000 for full training
    warmup_steps=1000,  # set to 2000 for full training
    overwrite_output_dir=True,
    num_train_epochs=epochs,
    save_total_limit=3,
    fp16=True, 
)

In [20]:
# Pass into the trainer
trainer = Seq2SeqTrainer(
    model=bert2bert,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Using amp fp16 backend


In [21]:
trainer.train()

***** Running training *****
  Num examples = 50000
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 12500
/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:337: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
2500,4.401000,4.320231,19.392600,3.193400,13.381300,13.389000
5000,3.807500,3.946222,23.177900,4.677500,15.700600,15.697800
7500,3.377000,3.737342,26.321700,5.996300,17.551800,17.549700
10000,3.176900,3.586982,27.887000,6.978300,18.503500,18.516200
12500,3.095600,3.513162,28.402300,7.331200,18.727200,18.738300


Saving model checkpoint to ./checkpoint-500
Configuration saved in ./checkpoint-500/config.json
Model weights saved in ./checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./checkpoint-500/tokenizer_config.json
Special tokens file saved in ./checkpoint-500/special_tokens_map.json
Saving model checkpoint to ./checkpoint-1000
/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:337: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "
Configuration saved in ./checkpoint-1000/config.json
Model weights saved in ./checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./checkpoint-1000/tokenizer_config.json
Special 

TrainOutput(global_step=12500, training_loss=3.71784671875, metrics={'train_runtime': 15966.5273, 'train_samples_per_second': 6.263, 'train_steps_per_second': 0.783, 'total_flos': 7.668176256e+16, 'train_loss': 3.71784671875, 'epoch': 2.0})

In [22]:
wandb.finish()

eval/loss,█▅▃▂▁
eval/rouge1,▁▄▆██
eval/rouge2,▁▄▆▇█
eval/rougeL,▁▄▆██
eval/rougeLsum,▁▄▆██
eval/runtime,█▆▃▅▁
eval/samples_per_second,▁▃▆▄█
eval/steps_per_second,▁▃▆▄█
train/epoch,▁▂▂▂▃▃▃▄▅▅▅▆▆▆▇███
train/global_step,▁▂▂▂▃▃▃▄▅▅▅▆▆▆▇███
train/learning_rate,█▇▇▆▅▅▄▄▃▂▂▁


## Results and Evaluation

In [23]:
!ls -lh

total 24K
drwxr-xr-x 2 root root 4.0K Nov 27 03:29 checkpoint-11500
drwxr-xr-x 2 root root 4.0K Nov 27 03:35 checkpoint-12000
drwxr-xr-x 2 root root 4.0K Nov 27 04:01 checkpoint-12500
drwxr-xr-x 3 root root 4.0K Nov 26 23:35 runs
drwxr-xr-x 1 root root 4.0K Nov 18 14:36 sample_data
drwxr-xr-x 3 root root 4.0K Nov 26 23:33 wandb


In [25]:
from transformers import BertTokenizer, EncoderDecoderModel

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = EncoderDecoderModel.from_pretrained("./checkpoint-12500")
model.to("cuda")

test_data = raw_datasets['test']

loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/bert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/534479488c54aeaf9c3406f647aa2ec13648c06771ffe269edabebd4c412da1d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc7043

In [26]:
# map data correctly
def generate_summary(batch):
    # Tokenizer will automatically set [BOS] <text> [EOS]
    # cut off at BERT max length 512
    inputs = tokenizer(batch["article"], padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")

    outputs = model.generate(input_ids, attention_mask=attention_mask)

    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str

    return batch

In [27]:

# only use 16 training examples for notebook - DELETE LINE FOR FULL TRAINING
test_data = test_data.select(range(16))

results = test_data.map(generate_summary, batched=True, batch_size=batch_size)

  0%|          | 0/2 [00:00<?, ?ba/s]

In [28]:
pred_str = results["pred"]
label_str = results["highlights"]

In [29]:
for i in range(16):
    print("Original Text: %s" % results[i]['article'])
    print("\nActual Summary: %s" % label_str[i])
    print("\nPredicted Summary: %s" % pred_str[i])
    print("=====================================================================\n")

Original Text: (CNN)James Best, best known for his portrayal of bumbling sheriff Rosco P. Coltrane on TV's "The Dukes of Hazzard," died Monday after a brief illness. He was 88. Best died in hospice in Hickory, North Carolina, of complications from pneumonia, said Steve Latshaw, a longtime friend and Hollywood colleague. Although he'd been a busy actor for decades in theater and in Hollywood, Best didn't become famous until 1979, when "The Dukes of Hazzard's" cornpone charms began beaming into millions of American homes almost every Friday night. For seven seasons, Best's Rosco P. Coltrane chased the moonshine-running Duke boys back and forth across the back roads of fictitious Hazzard County, Georgia, although his "hot pursuit" usually ended with him crashing his patrol car. Although Rosco was slow-witted and corrupt, Best gave him a childlike enthusiasm that got laughs and made him endearing. His character became known for his distinctive "kew-kew-kew" chuckle and for goofy catchphras